In [19]:
import sys
sys.path.append('../..')

from utils.normalization import normalizeAuditdTable
from utils.misc import getNotebookPath
from utils.preprocessing import readAndFilterFile
from utils.constants import *
import os
import pandas as pd

SCRIPT_PATH = getNotebookPath()

# input
DATA_FOLDER = SCRIPT_PATH + "\\..\\..\\data\\"
LIMIT = 10
files = os.listdir(DATA_FOLDER+"auditd_msft_raw\\")[:LIMIT]

out = []
for file in files:
    ldf = readAndFilterFile(DATA_FOLDER+"auditd_msft_raw\\"+file, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES})
    out.append(ldf)
out = pd.concat(out)
out.fillna("(none)", inplace=True)

df = normalizeAuditdTable(out.copy())

In [31]:
df['rule.sidid'].value_counts()

execution               286324
mount                    12327
external-access          11707
log-modification         11503
user_start                4558
user_end                  4533
bind-access               1813
perm_modification         1670
unsucces_file_access      1000
inbound-access             965
time-change                363
Name: rule.sidid, dtype: int64

In [40]:
groupByCols = ['hostname']#, 'process.ppid']
l = df.groupby(groupByCols).size().shape[0]
for i, (group, gdf) in enumerate(df.groupby(groupByCols)):
    #file = "_".join(group) + ".json"
    file = group + ".json"
    print(f"{i+1}/{l} -- {file}")
    with open(DATA_FOLDER+"auditd_msft_grouped\\"+file, "w") as f:
        f.write(gdf.drop(columns=groupByCols).to_json(orient='records', indent=4))
    if gdf['rule.sidid'].isin(['external-access']).any():
        print(f"[!!!] {file} contains external-access events")

1/2621 -- abhiag-prodnam.json
2/2621 -- aks-agentpool-10334287-vmss_1.json
3/2621 -- aks-agentpool-11935980-vmss_0.json
4/2621 -- aks-agentpool-11935980-vmss_1.json
5/2621 -- aks-agentpool-17867375-vmss_0.json
6/2621 -- aks-agentpool-19096168-vmss_0.json
7/2621 -- aks-agentpool-24256790-vmss_2.json
8/2621 -- aks-agentpool-25353137-vmss_0.json
9/2621 -- aks-agentpool-26280060-vmss_2.json
10/2621 -- aks-agentpool-28187160-vmss_1.json
11/2621 -- aks-agentpool-31873240-vmss_1.json
12/2621 -- aks-agentpool-35382139-vmss_2.json
13/2621 -- aks-agentpool-39110072-vmss_0.json
[!!!] aks-agentpool-39110072-vmss_0.json contains external-access events
14/2621 -- aks-agentpool-39110072-vmss_1.json
15/2621 -- aks-agentpool-39110072-vmss_2.json
16/2621 -- aks-agentpool-41692427-vmss_0.json
17/2621 -- aks-agentpool-59875483-vmss_1.json
18/2621 -- aks-agentpool-59875483-vmss_2.json
19/2621 -- aks-agentpool-77327031-vmss_2.json
20/2621 -- aks-agentpool-84407388-vmss_1.json
21/2621 -- aks-agentpool-844073

KeyboardInterrupt: 